# Ejercicio Práctico: Apriori Básico

## Estudiantes

Marco Piedra Venegas (A64397)

María Fernanda Vega (B78244)

Valeria Fonseca Rodríguez (B93061)


## Carga del dataset

Se trata de un dataset transaccional de compras en línea. Corresponde a un un mayorista de regalos en Reino Unido.

Fuente: Daqing Chen (2015). Online Retail. UCI Machine Learning Repository. https://doi.org/10.24432/C5BW33

In [13]:
# Paquete para carga del dataset
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo

In [14]:
# Obtener dataset del repositorio.
retail = fetch_ucirepo(id=352)

# Describir variables del dataset.
retail.variables

,name,role,type,demographic,description,units,missing_values
0,InvoiceNo,ID,Categorical,None,a 6-digit integral number uniquely assigned to...,None,no
1,StockCode,ID,Categorical,None,a 5-digit integral number uniquely assigned to...,None,no
2,Description,Feature,Categorical,None,product name,None,no
3,Quantity,Feature,Integer,None,the quantities of each product (item) per tran...,None,no
4,InvoiceDate,Feature,Date,None,the day and time when each transaction was gen...,None,no
5,UnitPrice,Feature,Continuous,None,product price per unit,sterling,no
6,CustomerID,Feature,Categorical,None,a 5-digit integral number uniquely assigned to...,None,no
7,Country,Feature,Categorical,None,the name of the country where each customer re...,None,no


## Preprocesamiento del dataset

Se realiza la limpieza, selección, y reducción del dataset.

Se seleccionan las transacciones con 10 ítems o menos

In [15]:
# Seleccionar columnas de ID de transacción y ID de ítem.
data = retail.data.original[['InvoiceNo', 'StockCode']]
data.columns = ['transaction', 'item']

# Normalizar identificadores, de modo que solo queden ítems con código numérico.
data['transaction'] = data['transaction'].str.lower()
data['item'] = data['item'].str.lower()

# Eliminar del dataset transacciones e ítems potencialmente no relevantes.
data = data[~data['transaction'].str.contains('[a-z]')]
data = data[~data['item'].str.contains('[a-z]')]
data = data.drop_duplicates()

# Mostrar dataset.
data

<ipython-input-15-25dbca0592e7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['transaction'] = data['transaction'].str.lower()
<ipython-input-15-25dbca0592e7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['item'] = data['item'].str.lower()


,transaction,item
1,536365,71053
5,536365,22752
6,536365,21730
7,536366,22633
8,536366,22632
...,...,...
541904,581587,22613
541905,581587,22899
541906,581587,23254
541907,581587,23255


In [16]:
# Explorar cantidad de ítems por transacción

transaction_items = data.groupby('transaction').nunique()
transaction_items.columns = ['itemcount']
transaction_items.describe()


,itemcount
count,21065.000000
mean,22.277712
std,40.302880
min,1.000000
25%,4.000000
50%,13.000000
75%,25.000000
max,984.000000


In [17]:
# Reducir dataset a transacciones con la mediana o menos de ítems

transaction_items = transaction_items[transaction_items['itemcount'] <= int(transaction_items.median())]
data = data[data['transaction'].isin(transaction_items.index)]
data

<ipython-input-17-1e97a823e875>:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  transaction_items = transaction_items[transaction_items['itemcount'] <= int(transaction_items.median())]


,transaction,item
1,536365,71053
5,536365,22752
6,536365,21730
7,536366,22633
8,536366,22632
...,...,...
541868,581584,85038
541890,581586,22061
541891,581586,23275
541892,581586,21217


## Algoritmo FP-Growth

Se implementa el algoritmo FP-Growth para la generación de reglas de asociación, utilizando la biblioteca `mlxtend` en Python. El algoritmo construye una estructura llamada FP-tree que permite encontrar patrones frecuentes de forma compacta y escalable.


Agrupar ítems por transacción para FP-Growth

In [18]:
transactions = data.groupby('transaction')['item'].apply(list)

# convertimos a la lista de listas que necesita TransactionEncoder:
transactions_list = transactions.tolist()

# ver 3 transacciones:
for t in transactions_list[:3]:
    print(t)

['71053', '22752', '21730']
['22633', '22632']
['84879', '22745', '22748', '22749', '22310', '84969', '22623', '22622', '21754', '21755', '21777', '48187']


One-hot encoding para transacciones

In [19]:
from mlxtend.preprocessing import TransactionEncoder
from pandas import DataFrame

te = TransactionEncoder()
te_ary = te.fit(transactions_list).transform(transactions_list)
df_encoded = DataFrame(te_ary, columns=te.columns_)

Implementación de FP-Growth

In [20]:
from mlxtend.frequent_patterns import fpgrowth

# Genera los itemsets frecuentes
frequent_itemsets = fpgrowth(df_encoded, min_support=0.005, use_colnames=True)

# Ver resultados
frequent_itemsets.sort_values('support', ascending=False).head()


,support,itemsets
70,0.047286,(22423)
206,0.036063,(47566)
4,0.033303,(84879)
248,0.026863,(23084)
121,0.024103,(22178)


Generar reglas de asociación

In [21]:
from mlxtend.frequent_patterns import association_rules

# Genera todas las reglas
rules = association_rules(frequent_itemsets,
                          metric="confidence",
                          min_threshold=0.0) # soporte mínimo 0 para que no descarte nada

# Ver las primeras reglas
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())


  antecedents consequents   support  confidence       lift
0     (82486)     (82483)  0.005428    0.427536  44.685758
1     (82483)     (82486)  0.005428    0.567308  44.685758
2     (22114)     (22112)  0.005704    0.469697  27.747859
3     (22112)     (22114)  0.005704    0.336957  27.747859
4     (22727)     (22726)  0.009936    0.600000  36.435754


Confianza total (support): muestra las reglas más frecuentes en el dataset.

Confianza máxima (confidence): muestra las reglas que, una vez que aparece A, tienen más probabilidad de que aparezca B.

In [22]:
simple_rules = rules[['antecedents', 'consequents', 'support', 'confidence']]

# filtra por Confianza Total (support)
top_support = simple_rules.sort_values('support', ascending=False).head(10)
print("10 con mayor Confianza Total (support):")
print(top_support)

# filtra por Confianza Máxima (confidence)
top_confidence = simple_rules.sort_values('confidence', ascending=False).head(10)
print("\n10 con mayor Confianza Máxima (confidence):")
print(top_confidence)


10 con mayor Confianza Total (support):
       antecedents consequents   support  confidence
15         (22697)     (22699)  0.012144    0.709677
14         (22699)     (22697)  0.012144    0.660000
33         (22697)     (22698)  0.010396    0.607527
32         (22698)     (22697)  0.010396    0.763514
5          (22726)     (22727)  0.009936    0.603352
4          (22727)     (22726)  0.009936    0.600000
35         (22698)     (22699)  0.009476    0.695946
34         (22699)     (22698)  0.009476    0.515000
39  (22699, 22697)     (22698)  0.008464    0.696970
38  (22699, 22698)     (22697)  0.008464    0.893204

10 con mayor Confianza Máxima (confidence):
       antecedents consequents   support  confidence
38  (22699, 22698)     (22697)  0.008464    0.893204
50         (23171)     (23170)  0.005152    0.835821
40  (22697, 22698)     (22699)  0.008464    0.814159
32         (22698)     (22697)  0.010396    0.763514
31         (21136)     (84879)  0.005336    0.763158
19  (22423, 22

## Conclusiones
Cuando ordenamos únicamente por **confianza total (support)**, los patrones fuertes son básicamente los pares más frecuentes en todo el dataset. Este ranking no nos dice nada sobre cuán predictivas son esas relaciones.

Si lo comparamos con el ranking de la **confianza máxima (confidence)** esto revela patrones muy predictivos que quedarían ocultos si solo miráramos la frecuencia

Definimos **patrones interesantes** fuertes aquellos que combinan un soporte razonable (≳ 0.8 %) con una confianza condicional alta (≳ 70 %).

Como estos:

    22697 → 22699 (1.214 % de soporte, 70.97 % de confidence)

    22698 → 22697 (1.040 %, 76.35 %)

    (22699,22698) → 22697 (0.846 %, 89.32 %)

    (22697,22698) → 22699 (0.846 %, 81.42 %)

Clasificamos **patrones engañosos** a las reglas con confiabilidades altas pero soportes muy bajos (≲ 0.6 %). Aunque son muy predictivas, su cobertura tan limitada las hace poco prácticas para recomendaciones a gran escala.

Como estos:

```
23171 → 23170 (83.58 % de confidence, 0.515 % de support)

21136 → 84879 (76.32 % y 0.534 %).
```

